In [1]:
%cd /net/projects/robeczech/checkpoints/eol-210223

/lnet/ms/projects/robeczech/checkpoints/eol-210223


In [2]:
# Simple tokenizer example

import os
import sys

from tokenizer import robeczech_tokenizer

tokenizer = robeczech_tokenizer.RobeCzechTokenizer("tokenizer")

for line in ["Praha je hlavní město České republiky."]:
    encoded = tokenizer.encode(line.strip())
    print(encoded)
    print("Decoding:", tokenizer.decode(encoded["input_ids"]))

{'input_ids': [0, 1016, 12, 599, 509, 314, 635, 32, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Decoding: Praha je hlavní město České republiky.


In [3]:
# Simple TF Usage
import numpy as np
import tensorflow as tf
import transformers

from tokenizer import robeczech_tokenizer

tokenizer = robeczech_tokenizer.RobeCzechTokenizer("tokenizer")

encoded_text = [tokenizer.encode("Praha je hlavní město České republiky")["input_ids"]]
encoded_text = np.array(encoded_text)
print(encoded_text)

# It is possible to use TFAutoModel or any other TFAutoModel* variant
model = transformers.TFAutoModelWithLMHead.from_pretrained("tf")

for mask in range(1, len(encoded_text[0]) - 1):
    original = int(encoded_text[0][mask])
    print("Original='{}'".format(tokenizer.decode([original], skip_special_tokens=False)))
    for masked in [0, 1]:
        if masked:
            encoded_text[0][mask] = tokenizer.mask_index

        [logits], = model(encoded_text)
        argsorted_logits = np.argsort(-logits.numpy(), axis=1)
        print(" ", *[tokenizer.decode([argsorted_logits[mask][i]], skip_special_tokens=False) for i in range(16)])
    encoded_text[0][mask] = original


[[   0 1016   12  599  509  314  635    4    2]]
Original='Praha'
  Praha Pražská  Praha Ostrava Pražský Olomouc Brno Jihlava Pražské Bratislava PRAHA Město Plzeň Vídeň Paříž To
  Praha Ostrava Brno Liberec Kladno Jihlava Pardubice Olomouc To Plzeň Opava Jablonec Co Město Karviná Budějovice
Original=' je'
   je  není  jsou  -  bude  JE  se  být  1  to  má  – je  bylo  2  v
   -  ,  jako  je  –  /  pro  :  1  za  a  4  má  na  Praha  3
Original=' hlavní'
   hlavní  Hlavní  ústřední  největší  hlavním  hlavního  okresní  horní  nejvyšší  celé  vrchní  významné  nejvýznamnější  krajské  centrální Hlavní
   hlavní  největší  krajské  centrální  Hlavní  ústřední  druhé  hlavním  první  nejkrásnější  okresní  hlavního  nejvyšší  jediné  nejbohatší  poslední
Original=' město'
   město  města  měst  v  Město  městem  centrum  do Město  to  městě  území  městské  m  sídlo  nádraží
   město  městem  města  měst  částí  stát  část  metropole  oblastí  bod  součástí  cíl  centrum  obcí  městy  reg

In [4]:
# Simple PyTorch usage
import numpy as np
import torch
import transformers

from tokenizer import robeczech_tokenizer

tokenizer = robeczech_tokenizer.RobeCzechTokenizer("tokenizer")

encoded_text = [tokenizer.encode("Praha je hlavní město České republiky")["input_ids"]]
encoded_text = torch.Tensor(encoded_text).to(torch.int64)
print(encoded_text)

# It is possible to use AutoModel or any other AutoModel* variant
model = transformers.AutoModelWithLMHead.from_pretrained("pytorch")

for mask in range(1, len(encoded_text[0]) - 1):
    original = int(encoded_text[0][mask])
    print("Original='{}'".format(tokenizer.decode([original], skip_special_tokens=False)))
    for masked in [0, 1]:
        if masked:
            encoded_text[0][mask] = tokenizer.mask_index

        [logits], = model(encoded_text)
        argsorted_logits = torch.argsort(logits, dim=1, descending = True).numpy()
        print(" ", *[tokenizer.decode([argsorted_logits[mask][i]], skip_special_tokens=False) for i in range(16)])
    encoded_text[0][mask] = original


tensor([[   0, 1016,   12,  599,  509,  314,  635,    4,    2]])
Original='Praha'
  Praha Pražská  Praha Ostrava Pražský Olomouc Brno Jihlava Pražské Bratislava PRAHA Město Plzeň Vídeň Paříž To
  Praha Ostrava Brno Liberec Kladno Jihlava Pardubice Olomouc To Plzeň Opava Jablonec Co Město Karviná Budějovice
Original=' je'
   je  není  jsou  -  bude  JE  se  být  1  to  má  – je  bylo  2  v
   -  ,  jako  je  –  /  pro  :  1  za  a  4  má  na  Praha  3
Original=' hlavní'
   hlavní  Hlavní  ústřední  největší  hlavním  hlavního  okresní  horní  nejvyšší  celé  vrchní  významné  nejvýznamnější  krajské  centrální Hlavní
   hlavní  největší  krajské  centrální  Hlavní  ústřední  druhé  hlavním  první  nejkrásnější  okresní  hlavního  nejvyšší  jediné  nejbohatší  poslední
Original=' město'
   město  města  měst  v  Město  městem  centrum  do Město  to  městě  území  městské  m  sídlo  nádraží
   město  městem  města  měst  částí  stát  část  metropole  oblastí  bod  součástí  cíl  centrum  